BY: **RIYA JOSHI**

EMAIL: riya.joshi@somaiya.edu



---


### **Basic idea behind SVM(Support Vector Machine) algorithm**:

*   SVM (Support Vector Machine) is a supervised machine learning algorithm that can be used for both classification and regression tasks.
*   The algorithm works by finding a hyperplane (or decision boundary) which have the following properties:
  - It creates separation between data points of two classes with a maximum margin
  - Its equation (w.x + b = 0) yields a value ≥ 1 for examples from +ve class and ≤-1 for examples from -ve class
  
* It finds this hyperplane by finding the optimal values w (weights/normal) and b (intercept) which define this hyperplane. The optimal values are found by minimizing a cost function. 

* The data points above or on the hyperplane are classified as class +1 and the points be low are classified as class -1. In other words, the equation (w.x+b=0) yields a value ≥ 1 for +ve class and ≤ -1 for -ve class.


### **Applications of SVM**:

* Face detection 
* Text and hypertext categorization 
* Classification of images 
* Bioinformatics
* Protein fold and remote homology detection 
* Handwriting recognition 
* Generalized predictive control(GPC) 





---




In [15]:
# Importing required libraries
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv('./smoke.csv') #importing dataset
df.head() # displaying first five records

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [17]:
# replacing null values with 0
df = df.fillna(value= 0)

In [18]:
# Shuffling dataset ( to remove any biases in the dataset if any )
df = df.sample(frac=1)

In [19]:
new_df=df[['UTC', 'Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2','Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0', 'NC2.5', 'CNT', 'Fire Alarm']]
new_df.head()

,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
24438,1654757769,-3.123,48.17,1306,407,12995,19404,938.798,2.10,2.18,14.43,2.250,0.051,24438,1
19480,1654752811,19.955,52.29,1243,411,12919,19429,938.699,1.54,1.60,10.57,1.648,0.037,19480,1
18390,1654751721,19.712,47.29,1230,404,12905,19418,938.728,1.82,1.89,12.51,1.951,0.044,18390,1
8408,1654741739,-4.648,53.20,343,400,13091,19926,939.360,0.31,0.33,2.15,0.336,0.008,8408,1
42825,1654779174,26.740,48.58,1111,400,12909,19456,938.715,1.72,1.79,11.87,1.851,0.042,17831,1


In [20]:
# converting type from object to string to be able to apply Label Encoder
new_df['Fire Alarm']=new_df['Fire Alarm'].astype(dtype='string',copy=True)

In [21]:
# converting catagorical column 'Fire Alarm' to numerical column by LabelEncoding
new_df['Fire Alarm'] = LabelEncoder().fit_transform(new_df['Fire Alarm'])

In [22]:
# changing data type of whole dataset to int
new_df = new_df.astype(int)

In [23]:
# splitting dataset into 70:30 ratio

# Defining train size
train_size = int(0.7 * len(new_df))

# Splitting dataset
train_set = new_df[:train_size]
test_set = new_df[train_size:]

In [24]:
# separating train_set into X and Y
X_train=train_set.drop('Fire Alarm', axis=1)
y_train=train_set['Fire Alarm']

# separating test_set into X and Y
X_test=test_set.drop('Fire Alarm', axis=1)
y_test=test_set['Fire Alarm']

In [25]:
class SVM:

	def __init__(self, kernel = "linear", learning_rate = 1e-4, regularization_strength = 1.0, max_iter = 2000):
 
		self.num_feats = int
		self.train_size = int
		self.weights = np.array 
		self.y_train = np.array 
		self.input_matrix = np.array

		self.kernel = kernel
		self.kernel_matrix = np.array
		self.support_vectors = np.array
		self.learning_rate = learning_rate 	# Learning rate for gradient descent
		self.regularization_strength = regularization_strength 	# Regularization parameter, to control bias-variance tradeoff
		self.max_iter = max_iter	# Maximum Number of iterations to run gradient descent
		self.cost_threshold = 0.1 * learning_rate  # stopping criterion for gradient descent

	def fit(self, X, y):

		""" Adjust weights to training data """

		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]
		self.input_matrix = np.append(X, np.ones(self.train_size).reshape(-1, 1), axis = 1)   #Add Column with Ones for intercept term 
		self.y_train = np.where(y == 0, -1, 1)
		self.weights = np.zeros(self.num_feats + 1) #Extra +1 for the intercept

		# optimize weights
		prev_cost = float("inf")
		for i in range(self.max_iter):
			cost = self._update_weights()
			
			if i%100 ==0 or i == self.max_iter:
				print("Cost after {} iterations is: {}".format(i, cost))
			if abs(prev_cost -cost) < self.cost_threshold*prev_cost:
				print("Cost after {} iterations is: {}".format(i, cost))
				break
			prev_cost = cost

	def _update_weights(self):

		"""
			Cost Function:
				l(w) = sum(max(0, 1 - y(wX + b))) + (lambda/2)(||w||)^2
				First Term is Hinge Loss, Second is Regularization term
			Gradient:
			    delta_w = dl/dw = (1/n) * ( if y(wX+b) < 1: -yX + lambda*w else: lambda*w )
			Gradient Descent
				w = w - (learning_rate * delta_w)
		"""
		y_pred = (self.weights * self.input_matrix).sum(axis = 1) # y_pred = wX+b


		dist = (1 - (self.y_train * y_pred))
		dist[dist < 0] = 0
		hinge_loss = np.sum(dist)/self.train_size
		regularization_term = (1/2) * self.regularization_strength * (np.dot(self.weights, self.weights))
		cost = hinge_loss  + regularization_term

		delta_w =  self.regularization_strength * (self.weights)

		for y, yhat, X in zip(self.y_train, y_pred, self.input_matrix):

			if y*yhat < 1:
				delta_w -= y*X

		delta_w /= self.train_size
		self.weights = self.weights - (self.learning_rate * delta_w)

		return cost

	def predict(self, X):

		""" Make predictions on given X using trained model """

		size = X.shape[0]
		X = np.append(X, np.ones(size).reshape(-1, 1), axis = 1)

		y_pred = np.sign((self.weights * X).sum(axis = 1))

		y_pred[np.where(y_pred == -1)] = 0.0

		return y_pred 

In [26]:
svm_clf = SVM(learning_rate = 1e-5, regularization_strength = 10.5)

svm_clf.fit(X_train, y_train)

Cost after 0 iterations is: 1.0
Cost after 100 iterations is: 1663357175255.456
Cost after 200 iterations is: 5551815682561.569
Cost after 300 iterations is: 1634179982000.7266
Cost after 400 iterations is: 5522633142537.887
Cost after 500 iterations is: 1605002824457.9653
Cost after 600 iterations is: 5493450638225.304
Cost after 700 iterations is: 1575825702627.2136
Cost after 800 iterations is: 5464268169623.829
Cost after 900 iterations is: 1546648616508.481
Cost after 1000 iterations is: 5435085736733.499
Cost after 1100 iterations is: 1517471584795.6877
Cost after 1200 iterations is: 5405903358251.653
Cost after 1300 iterations is: 1488294588794.9038
Cost after 1400 iterations is: 5376721015480.932
Cost after 1500 iterations is: 1459117628506.141
Cost after 1600 iterations is: 5347538708421.34
Cost after 1700 iterations is: 1429940703929.4302
Cost after 1800 iterations is: 5318356437072.923
Cost after 1900 iterations is: 1400763833758.5771


In [27]:
print("Coefficients: {}".format(svm_clf.weights[:-1]))

Coefficients: [ 1.12128754e+04 -2.10285221e-02  3.27671783e-02 -1.55641265e+01
 -1.74887714e+00  3.63828631e-01 -1.69263298e+00  9.46116429e-03
 -1.00484911e+00 -1.68143278e+00 -5.31510104e+00 -1.84088708e+00
 -6.42969431e-01  4.62129491e+01]


In [28]:
print("Intercept: {}".format(svm_clf.weights[-1]))

Intercept: 7.0062162723936404e-06


In [29]:
def accuracy_score(y_true, y_pred):

	"""	score = (y_true - y_pred) / len(y_true) """

	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100 ,2)

In [30]:
print("Train Accuracy: {}".format(accuracy_score(y_train, svm_clf.predict(X_train))))
print("Test Accuracy: {}".format(accuracy_score(y_test, svm_clf.predict(X_test))))

Train Accuracy: 71.5
Test Accuracy: 71.38
